# Challenge Amadeus
1. Count lines
At first we hace to open the file bookings.csv.bz2

In [220]:
import pandas as pd
import numpy as np
import matplotlib as plot
import bz2

My file is in /home/dsc/Data/challenge/bookings.csv.bz2

In [221]:
pFolder = '/home/dsc/Data/challenge/'
fBookings = pFolder+'bookings.csv.bz2'
fSearches = pFolder+'searches.csv.bz2'

I try open the file with read_csv and create a DataFrame for the other exercices

In [222]:
#dataBookings = pd.read_csv(fBookings,delimiter='^',iterator=True,chunksize=100000)


In [223]:
#count=0
#for data in dataBookings:
#    count +=1
#print(count)

KeyboardInterrupt: 

In [224]:
#dfBookings = pd.concat(dataBookings)

KeyboardInterrupt: 

In [226]:
count = 0
with bz2.BZ2File(fBookings, "r") as bz:
    for line in bz:
        count += 1
print(count)

10000011


In [ ]:
count = 0
with bz2.BZ2File(fSearches, "r") as bz:
    for line in bz:
        count += 1
print(count)

2. Top 10 arrival airports in the world in 2013 of bookings file

We will create a sample of bookings and we can play confortable

In [ ]:
! bzcat {pFolder}bookings.csv.bz2 | head -100000 | bzip2 -c > {pFolder}bookings.sample.csv.bz2 

In [ ]:
! bzcat {pFolder}bookings.sample.csv.bz2 | wc 

In [ ]:
fSample = pFolder + 'bookings.sample.csv.bz2'

In [ ]:
df_sample = pd.read_csv(fSample,delimiter='^')
df_sample.sample(5)

In [ ]:
df_sample.columns=df_sample.columns.str.replace('\s+', '')

In [ ]:
df_sample_sol= df_sample[['arr_port','pax','year']]

In [ ]:
df_sample_sol=df_sample_sol[df_sample_sol['year']==2013]
df_sample_sol= df_sample[['arr_port','pax']]
df_sample_sol['arr_port']= df_sample['arr_port'].str.strip()

In [ ]:
df_sample_sol.groupby(['arr_port']).sum().sort_values(by='pax',ascending=False).head(10)

Note that we have arr_port with negative sum of pax, it's only a sampel and we can lost the good paxes, but if we find this in the full file maybe we have a bad data.

Now we have the exercice 2 solve to the sample, our new target is open the full file.

In [ ]:
delimiter='^'
chunksize=500000
usecols=['arr_port','pax','year']
dataBookings = pd.read_csv(fBookings, delimiter=delimiter, compression='bz2', iterator=True, chunksize=chunksize,usecols=usecols)

In [ ]:
df_total=pd.DataFrame()
for chunk in dataBookings:
    df=chunk[chunk['year']==2013]
    df=df[['arr_port','pax']]
    df['arr_port']=df['arr_port'].str.strip().str.upper()
    df=df.groupby('arr_port').sum().sort_values(by='pax',ascending=False)
    df_total=df_total.append(df)

In [ ]:
df_total.groupby('arr_port').sum().sort_values(by='pax',ascending=False).head(10)